# Data Analysis

### Import necessary libriries 

In [2]:
import pandas as pd

### Merge bts table with medictions table
It's necessary to merge for test and train datasets. Also these datasets must be reduced to correctly join bts dataset

In [8]:
bts = pd.read_csv('./data/bts.csv')
meds = pd.read_csv('./data/meds_train.csv')

meds_train_melt = pd.melt(
    meds,
    id_vars=['ponto_id', 'delay_1', 'delay_2', 'delay_3', 'lat', 'lon', 'indoor'],
    value_vars=['rssi_1_1', 'rssi_1_2', 'rssi_1_3', 'rssi_2_1', 'rssi_2_2', 'rssi_2_3', 'rssi_3_1', 'rssi_3_2', 'rssi_3_3'],
    var_name='rssi_id',
    value_name='rssi_value'
)

# merge tables
merged_df = pd.merge(
    bts,
    meds_train_melt,
    left_on='RssiId',
    right_on='rssi_id',
)

# save to csv
merged_df.to_csv('./data/data_train.csv', index=False)

In [10]:
# same thing for test data
meds_test = pd.read_csv('./data/meds_test.csv')

meds_test_melt = pd.melt(
    meds_test,
    id_vars=['ponto_id', 'delay_1', 'delay_2', 'delay_3'],
    value_vars=['rssi_1_1', 'rssi_1_2', 'rssi_1_3', 'rssi_2_1', 'rssi_2_2', 'rssi_2_3', 'rssi_3_1', 'rssi_3_2', 'rssi_3_3'],
    var_name='rssi_id',
    value_name='rssi_value'
)

# merge tables
merged_df_test = pd.merge(
    bts,
    meds_test_melt,
    left_on='RssiId',
    right_on='rssi_id',
)

# save to csv
merged_df_test.to_csv('./data/data_test.csv', index=False)